## 1. From epidemic to pandemic
<p><img style="float: left; margin:5px 20px 5px 1px; width:40%" src="https://www.nps.gov/aboutus/news/images/CDC-coronavirus-image-23311-for-web.jpg?maxwidth=650&autorotate=false"></p>
<p>In December 2019, COVID-19 coronavirus was first identified in the Wuhan region of China. By March 11, 2020, the World Health Organization (WHO) categorized the COVID-19 outbreak as a pandemic. A lot has happened in the months in between with major outbreaks in Iran, South Korea, and Italy. </p>
<p>We know that COVID-19 spreads through respiratory droplets, such as through coughing, sneezing, or speaking. But, how quickly did the virus spread across the globe? And, can we see any effect from country-wide policies, like shutdowns and quarantines? </p>
<p>Fortunately, organizations around the world have been collecting data so that governments can monitor and learn from this pandemic. Notably, the Johns Hopkins University Center for Systems Science and Engineering created a <a href="https://github.com/RamiKrispin/coronavirus">publicly available data repository</a> to consolidate this data from sources like the WHO, the Centers for Disease Control and Prevention (CDC), and the Ministry of Health from multiple countries.</p>
<p>In this notebook, you will visualize COVID-19 data from the first several weeks of the outbreak to see at what point this virus became a global pandemic.</p>
<p><em>Please note that information and data regarding COVID-19 is frequently being updated. The data used in this project was pulled on March 17, 2020, and should not be considered to be the most up to date data available.</em></p>

In [1]:
### pip(optionally) or conda-forge/packages installation on local machine and/or on the cloud (i.e., IBM Watson) 
## A Jupyter notebook extension to hide code and prompts - https://pypi.org/project/hide-code/0.2.0/
## File type identification using libmagic
# !pip install python-magic==0.4.18
## Easily download, build, install, upgrade, and uninstall Python packages
# !pip install setuptools==46.3.1
## A built-package format for Python
# !pip install wheel
# !pip install pixiedust==1.1.18
# !pip install --user --upgrade pixiedust
# !pip install rosie==1.1.4
# !pip install pixiedust_rosie
# !pip install hide-code==0.5.5
## for formatting purposes.
# !conda update nbformat
## Generic syntax highlighter suitable for use in code hosting, forums, wikis or other applications that need to prettify source code
# !conda install -c anaconda pygments 
## provides a command line tool to convert Jupyter notebook files to HTML files - https://www.nbinteract.com/
# !pip install nbinteract
## An interactive, browser-based graphing library for Python
# !conda install -c conda-forge plotly
## Jupyter Interactive Widgets
# !conda install -c conda-forge ipywidgets
## Make beautiful maps with Leaflet.js and Python
# !conda install -c conda-forge folium
## A Jupyter extensions that turns notebooks into web applications. - https://github.com/oschuett/appmode
# !pip install appmode
## Rendering of live Jupyter notebooks with interactive widgets - https://anaconda.org/conda-forge/voila
# !conda install -c conda-forge voila
## Gridstack template for voila - https://anaconda.org/conda-forge/voila-gridstack
# !conda install -c conda-forge voila-gridstack
## Vuetify specific template for Voila - https://anaconda.org/conda-forge/voila-vuetify
# !conda install -c conda-forge voila-vuetify
## Reveal template for voila - https://anaconda.org/conda-forge/voila-reveal
# !conda install -c conda-forge voila-reveal
## Material template for Voila - https://anaconda.org/conda-forge/voila-material
# !conda install -c conda-forge voila-material
## A collection of various different notebook extensions for Jupyter - https://anaconda.org/conda-forge/jupyter_contrib_nbextensions
# !conda install -c conda-forge jupyter_contrib_nbextensions
# !easy_install ibm_db
# !pip install ibm-db==3.0.1
# !pip install ijson
# A python native weaviate client makes using the Weaviate API easier.
# !pip install weaviate-client

In [2]:
# @hidden_cell
# import packages
from __future__ import print_function
import ibm_db
import ibm_db_dbi
import pandas as pd
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from IPython.core.display import display, HTML

import pixiedust
# london_info = pixiedust.sampleData("https://files.datapress.com/
# london/dataset/london-borough-profiles/2015-09-24T15:50:01/Londonborough-
# profiles.zip")
import pixiedust_rosie
# pixiedust_rosie.wrangle_data()

# https://www.pluralsight.com/guides/web-scraping-with-request-python
import requests as reqs # To use request package with alliance(s) in current program 
from requests.exceptions import HTTPError

# import ijson
import json
import time

import weaviate

Pixiedust database opened successfully


In [3]:
# url = "https://raw.githubusercontent.com/pixiedust/pixiedust_rosie/master/sample-data/HCUP_Tutorial.csv"
# url = "https://github.com/ibm-watson-data-lab/pixiedust_rosie/raw/master/sample-data/Healthcare_Cost_and_Utilization_Project__HCUP__-_National_Inpatient_Sample.csv"
url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
pixiedust_rosie.wrangle_data(url)

Downloaded 169274 bytes
Creating pandas DataFrame for 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'. Please wait...
Successfully created pandas DataFrame for 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'


RuntimeError: Cannot find librosie in local directory or system installation

In [ ]:
# DON'T TOUCH THE 'DRIVER' ATTRIBUTE!
dsn = "DATABASE=BLUDB;HOSTNAME=dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net;PORT=50000;PROTOCOL=TCPIP;UID=hbs51528;PWD=hl9zvb3t4sp0w6@r;"
# dsn = "DRIVER={{IBM DB2 ODBC DRIVER}};" + \
#       "DATABASE={BLUDB};" + \
#       "HOSTNAME={dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net};" + \
#       "PORT=50000;" + \
#       "PROTOCOL=TCPIP;" + \
#       "UID={hbs51528};" + \
#       "PWD={hl9zvb3t4sp0w6@r};"
SCHEMA_NAME = 'HBS51528'
hdbc = ibm_db.connect(dsn, "", "")
hdbi = ibm_db_dbi.Connection(hdbc)
print('Connection To DB2 Instance Has Been Created!')

In [ ]:
# Query Strings
sql = 'CREATE TABLE '+SCHEMA_NAME+'.JHUCOVIDSUMMARY ( ' + \
      'ID SMALLINT NOT NULL GENERATED ALWAYS AS IDENTITY, ' + \
      'Global__NewConfirmed INTEGER, ' + \
      'Global__TotalConfirmed INTEGER, ' + \
      'Global__NewDeaths INTEGER, ' + \
      'Global__TotalDeaths INTEGER, ' + \
      'Global__NewRecovered INTEGER, ' + \
      'Global__TotalRecovered INTEGER, ' + \
      'Countries__Country VARCHAR(255), ' + \
      'Countries__CountryCode VARCHAR(255), ' + \
      'Countries__Slug VARCHAR(255), ' + \
      'Countries__NewConfirmed INTEGER, ' + \
      'Countries__TotalConfirmed INTEGER, ' + \
      'Countries__NewDeaths INTEGER, ' + \
      'Countries__TotalDeaths INTEGER, ' + \
      'Countries__NewRecovered INTEGER, ' + \
      'Countries__TotalRecovered INTEGER, ' + \
      'Countries__Date VARCHAR(255))'
#     var selectString = "select * from covidsummary";
#     var deleteString = "drop table covidsummaryUPDATE";

In [ ]:
try:
    stmt = ibm_db.exec_immediate(hdbc, sql)
    print('Table JHUCOVIDSUMMARY Has Been Created Under ' + str(SCHEMA_NAME) + ' Schema!')
except:
    print('ERROR: Table Already Exist Or Some Other Error')

In [ ]:
# sql2 = 'CREATE TABLE '+SCHEMA_NAME+'.JSONPO( ' + \
# 'ID SMALLINT NOT NULL GENERATED ALWAYS AS IDENTITY)'

In [ ]:
sql_create = 'CREATE TABLE '+SCHEMA_NAME+'.JSONPO( ' + \
        'ID INTEGER NOT NULL GENERATED ALWAYS AS IDENTITY, DATA BLOB(16M) INLINE LENGTH 25000, PRIMARY KEY(ID)) CCSID UNICODE;' 

In [ ]:
try:
    stmt = ibm_db.exec_immediate(hdbc, sql_create)
    print('Table JSONPO Has Been Created Under ' + str(SCHEMA_NAME) + ' Schema!')
except:
    print('ERROR: Table Already Exist Or Some Other Error')

In [ ]:
# Using a switcher (or a dictionary to map cases to their functionality) for Python Switch Case Statement
def dict_key(key):
        switcher={
                'Global':'Global',
                'Countries':'Countries'
             }
        return switcher.get(key,"Not a valid key")

In [ ]:
# # Using Python Functions & Lambdas in the dictionary (dict)
# def Global():
#         return 'Global'
# def Countries():
#         return 'Countries'
# def indirect(key):
#         switcher={
#                 'Global':Global,
#                 'Countries':Countries
#                 }
#         func=switcher.get(key,'Not a valid key')
#         return func()

In [ ]:
#Parse JSON (HAD TO USE API URL)
def jhu_search():
    try:
#         url = 'https://api.covid19api.com/summary#'
        url = 'https://covid19.mathdro.id/api'
        # To execute get request
        response =  reqs.get(url) # Difference between urrlib & request: https://stackoverflow.com/questions/38114499/difference-between-requests-get-and-urrlib-request-urlopen-python
        response.raise_for_status() # Raise error in case of failure
        jsonResponse = response.json() # To get response dictionary as JSON content             
        
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except requests.exceptions.ConnectionError as connErr: 
        print ("Error Connecting:", connErr) 
    except requests.exceptions.Timeout as timeOutErr: 
        print ("Timeout Error:", timeOutErr) 
    except requests.exceptions.RequestException as reqErr: 
        print ("Something Else:", reqErr) 
    except OSError as exc:
        if exc.errno == 36:
            print(f'OSError: [Errno 36] File name too long: {exc}')
        else:
            raise  # re-raise previously caught exception
    except Exception as err:
        print(f'Other error occurred: {err}')
        
               
#     return jsonResponse

# # iterate all JSON key-value pairs one-by-one.

# print('Your IP is {0}'.format(response.json()['origin']))

    for hlKey in jsonResponse:
        if dict_key(hlKey) == 'Global':
            print('A standard dictionary (dict) key: {0}'.format(hlKey))
        else:
            print('Other key: {0}'.format(hlKey))
        for key, value in jsonResponse.items():
            if isinstance(value, dict):
                print('Likely key with a list of multiple values in the dict : {0}'.format(value))
            else:
                print('A standard dict key: {0} & value: {1}'.format(key,value))
        
#         for k, v in jsonResponse.items():
# #             time.sleep(0.05)
#             for k1, v1 in v.items():
#                 print(k1)       
        
# https://www.dataquest.io/blog/python-json-tutorial/
    

In [ ]:
jhu_search()

In [ ]:
# try:
#     #Parse JSON (HAD TO USE API URL)
# #     url = "https://api.covid19api.com/summary#"
# #     json_obj = requests.get(url)
# #     data = json_obj.json()
#         url = 'https://api.covid19api.com/summary#'
#         # To execute get request
#         response =  reqs.get(url) # Difference between urrlib & request: https://stackoverflow.com/questions/38114499/difference-between-requests-get-and-urrlib-request-urlopen-python
#         response.raise_for_status() # Raise error in case of failure
#         data = response.json() # To get response dictionary as JSON content
        
#         for i in jsonResponse:
#             sql_inter = 'INSERT INTO '+SCHEMA_NAME+'.JSONPO VALUES ('
#             'SYSTOOLS.JSON2BSON(\' {"NewConfirmed": "'+str(data[i]['Global']['NewConfirmed'])+'"} \') , ' # SYSTOOLS.JSON2BSON({\''+data[i]+'\'}))'
#             stmt = ibm_db.exec_immediate(hdbc, sql_inter)
#             print('Table JSONPO Has Been Created Under ' + str(SCHEMA_NAME) + ' Schema!')
# except:
#     print('ERROR: Table Already Exist Or Some Other Error')

In [ ]:
# # Global and Countries
# # 'ID, ' + \
# sql_p1 = 'INSERT INTO '+SCHEMA_NAME+'.JHUCOVIDSUMMARY (' + \
#             'Global__NewConfirmed' + ', ' + \
#             'Global__TotalConfirmed' + ', ' + \
#             'Global__NewDeaths' + ', ' + \
#             'Global__TotalDeaths' + ', ' + \
#             'Global__NewRecovered' + ', ' + \
#             'Global__TotalRecovered' + ', ' + \
#             'Countries__Country' + ', ' + \
#             'Countries__CountryCode' + ', ' + \
#             'Countries__Slug' + ', ' + \
#             'Countries__NewConfirmed' + ', ' + \
#             'Countries__TotalConfirmed' + ', ' + \
#             'Countries__NewDeaths' + ', ' + \
#             'Countries__TotalDeaths' + ', ' + \
#             'Countries__NewRecovered' + ', ' + \
#             'Countries__TotalRecovered' + ', ' + \
#             'Countries__Date' + ')'

# #Parse JSON (HAD TO USE API URL)
# def jhu_search():
#     url = "https://api.covid19api.com/summary#"
#     json_obj = requests.get(url)
#     data = json_obj.json()          
# #     return data
#     sql_p2 = ''
#     for gItem in data['Global']:
#         for cItem in data['Countries']:
#             sql_p2 += ' VALUES ('
#             'SYSTOOLS.JSON2BSON(\' {"NewConfirmed": "'+str(gItem['Global']['NewConfirmed'])+'"} \') , ' 
#             'SYSTOOLS.JSON2BSON(\' {"TotalConfirmed": "'+str(gItem['Global']['TotalConfirmed'])+'"} \') , '
#             'SYSTOOLS.JSON2BSON(\' {"NewDeaths": "'+str(gItem['Global']['NewDeaths'])+'"} \') , '
#             'SYSTOOLS.JSON2BSON(\' {"TotalDeaths": "'+str(gItem['Global']['TotalDeaths'])+'"} \') , ' 
#             'SYSTOOLS.JSON2BSON(\' {"NewRecovered": "'+str(gItemm['Global']['NewRecovered'])+'"} \') , '
#             'SYSTOOLS.JSON2BSON(\' {"TotalRecovered": "'+str(gItem['Global']['TotalRecovered'])+'"} \') , ' 
#             'SYSTOOLS.JSON2BSON(\' {"Country": "'+str(cItem['Countries']['Country'])+'"} \') , '
#             'SYSTOOLS.JSON2BSON(\' {"CountryCode": "'+str(cItem['Countries']['CountryCode'])+'"} \') , '
#             'SYSTOOLS.JSON2BSON(\' {"Slug": "'+str(cItem['Countries']['Slug'])+'"} \') , '
#             'SYSTOOLS.JSON2BSON(\' {"NewConfirmed": "'+str(cItem['Countries']['NewConfirmed'])+'"} \') , '
#             'SYSTOOLS.JSON2BSON(\' {"TotalConfirmed": "'+str(cItem['Countries']['TotalConfirmed'])+'"} \') , '
#             'SYSTOOLS.JSON2BSON(\' {"NewDeaths": "'+str(cItem['Countries']['NewDeaths'])+'"} \') , '
#             'SYSTOOLS.JSON2BSON(\' {"TotalDeaths": "'+str(cItem['Countries']['TotalDeaths'])+'"} \') , '
#             'SYSTOOLS.JSON2BSON(\' {"NewRecovered": "'+str(cItem['Countries']['NewRecovered'])+'"} \') , '
#             'SYSTOOLS.JSON2BSON(\' {"TotalRecovered": "'+str(cItem['Countries']['TotalRecovered'])+'"} \') , '
#             'SYSTOOLS.JSON2BSON(\' {"Date": "'+str(cItem['Countries']['Date'])+'"} \'), '
#             ')'
            
#     sql_final  = sql_p1 + sql_p2
#     stmt = ibm_db.exec_immediate(hdbc, sql_final)

In [ ]:
# Global and Countries
# 'ID, ' + \
# sql_p1 = 'INSERT INTO '+SCHEMA_NAME+'.JHUCOVIDSUMMARY (' + \
#             'Global__NewConfirmed' + ', ' + \
#             'Global__TotalConfirmed' + ', ' + \
#             'Global__NewDeaths' + ', ' + \
#             'Global__TotalDeaths' + ', ' + \
#             'Global__NewRecovered' + ', ' + \
#             'Global__TotalRecovered' + ', ' + \
#             'Countries__Country' + ', ' + \
#             'Countries__CountryCode' + ', ' + \
#             'Countries__Slug' + ', ' + \
#             'Countries__NewConfirmed' + ', ' + \
#             'Countries__TotalConfirmed' + ', ' + \
#             'Countries__NewDeaths' + ', ' + \
#             'Countries__TotalDeaths' + ', ' + \
#             'Countries__NewRecovered' + ', ' + \
#             'Countries__TotalRecovered' + ', ' + \
#             'Countries__Date' + ')'

# #Store Analyzed Data from JSON API URL
# lam = data['Global']['NewDeaths']
# size = data['Global']['TotalDeaths']

In [ ]:
# for v, k in data.items():
#     if not isinstance(k, list):
#         for x, y in k.items():
#             print(x,':', y)
#     else:
#         for i in k:
#             for s, m in i.items():
#                 print(s,':', m)

# for item in data:
# sql_p2 = ' VALUES (' + 'SYSTOOLS.JSON2BSON(\' {"NewConfirmed": "'+str(item['Global']['NewConfirmed'])+'"} \'),'
# +        'SYSTOOLS.JSON2BSON(\' {"TotalConfirmed": "'+str(item['Global']['TotalConfirmed'])+'"} \'),'
# +        'SYSTOOLS.JSON2BSON(\' {"NewDeaths": "'+str(item['Global']['NewDeaths'])+'"} \'), '
# +        'SYSTOOLS.JSON2BSON(\' {"TotalDeaths": "'+str(item['Global']['TotalDeaths'])+'"} \'),'
# +        'SYSTOOLS.JSON2BSON(\' {"NewRecovered": "'+str(item['Global']['NewRecovered'])+'"} \'),'
# +        'SYSTOOLS.JSON2BSON(\' {"TotalRecovered": "'+str(item['Global']['TotalRecovered'])+'"} \'),' 
# +        'SYSTOOLS.JSON2BSON(\' {"Country": "'+str(item['Countries']['Country'])+'"} \'), '
# +        'SYSTOOLS.JSON2BSON(\' {"CountryCode": "'+str(item['Countries']['CountryCode'])+'"} \'),'
# +        'SYSTOOLS.JSON2BSON(\' {"Slug": "'+str(item['Countries']['Slug'])+'"} \'), '
# +        'SYSTOOLS.JSON2BSON(\' {"NewConfirmed": "'+str(item['Countries']['NewConfirmed'])+'"} \'),'
# +        'SYSTOOLS.JSON2BSON(\' {"TotalConfirmed": "'+str(item['Countries']['TotalConfirmed'])+'"} \'),'
# +        'SYSTOOLS.JSON2BSON(\' {"NewDeaths": "'+str(item['Countries']['NewDeaths'])+'"} \'),'
# +        'SYSTOOLS.JSON2BSON(\' {"TotalDeaths": "'+str(item['Countries']['TotalDeaths'])+'"} \'),'
# +        'SYSTOOLS.JSON2BSON(\' {"NewRecovered": "'+str(item['Countries']['NewRecovered'])+'"} \'),'
# +        'SYSTOOLS.JSON2BSON(\' {"TotalRecovered": "'+str(item['Countries']['TotalRecovered'])+'"} \'),'
# +        'SYSTOOLS.JSON2BSON(\' {"Date": "'+str(item['Countries']['Date'])+'"} \'),'
# +')'

# sql_final  = sql_p1 + sql_p2
# stmt = ibm_db.exec_immediate(hdbc, sql_final)


# global_newConfirmed = row['Global']['NewConfirmed']
# global_totalConfirmed = row['Global']['TotalConfirmed']
# global_newDeaths = row['Global']['NewDeaths']
# global_totalDeaths = row['Global']['TotalDeaths']
# global_newRecovered = row['Global']['NewRecovered']
# global_totalRecovered = row['Global']['TotalRecovered']
# countries_country = row['Countries']['Country']
# countries_countryCode = row['Countries']['CountryCode']
# countries_slug = row['Countries']['Slug']
# countries_newConfirmed = row['Countries']['NewConfirmed']
# countries_totalConfirmed = row['Countries']['TotalConfirmed']
# countries_newDeaths = row['Countries']['NewDeaths']
# countries_totalDeaths = row['Countries']['TotalDeaths']
# countries_newRecovered = row['Countries']['NewRecovered']
# countries_totalRecovered = row['Countries']['TotalRecovered']
# countries_date = row['Countries']['Date']
#     for index, row in data.iterrows():
#     sql_p2 = ' VALUES (' + \
#         'SYSTOOLS.JSON2BSON(\' {"NewConfirmed": "'+str(global_newConfirmed)+'"} \') , ' + \
#         'SYSTOOLS.JSON2BSON(\' {"TotalConfirmed": "'+str(global_totalConfirmed)+'"} \') , ' + \ 
#         'SYSTOOLS.JSON2BSON(\' {"NewDeaths": "'+str(global_newDeaths)+'"} \') , ' + \
#         'SYSTOOLS.JSON2BSON(\' {"TotalDeaths": "'+str(global_totalDeaths)+'"} \') , ' + \
#         'SYSTOOLS.JSON2BSON(\' {"NewRecovered": "'+str(global_totalConfirmed)+'"} \') , ' + \ 
#         'SYSTOOLS.JSON2BSON(\' {"TotalRecovered": "'+str(global_totalConfirmed)+'"} \') , ' + \ 
#         'SYSTOOLS.JSON2BSON(\' {"Country": "'+str(countries_country)+'"} \') , ' + \
#         'SYSTOOLS.JSON2BSON(\' {"CountryCode": "'+str(countries_countryCode)+'"} \') , ' + \
#         'SYSTOOLS.JSON2BSON(\' {"Slug": "'+str(countries_slug)+'"} \') , ' + \
#         'SYSTOOLS.JSON2BSON(\' {"NewConfirmed": "'+str(countries_newConfirmed)+'"} \') , ' + \
#         'SYSTOOLS.JSON2BSON(\' {"TotalConfirmed": "'+str(countries_totalConfirmed)+'"} \') , ' + \
#         'SYSTOOLS.JSON2BSON(\' {"NewDeaths": "'+str(countries_newDeaths)+'"} \') , ' + \
#         'SYSTOOLS.JSON2BSON(\' {"TotalDeaths": "'+str(countries_totalDeaths)+'"} \') , ' + \ 
#         'SYSTOOLS.JSON2BSON(\' {"NewRecovered": "'+str(countries_newRecovered)+'"} \') , ' + \
#         'SYSTOOLS.JSON2BSON(\' {"TotalRecovered": "'+str(countries_totalRecovered)+'"} \') , ' + \
#         'SYSTOOLS.JSON2BSON(\' {"Date": "'+str(countries_date)+' ' + \
#     ')'



In [ ]:
# Data Loading 
# Johns Hopkins CSV interaction using JHU CSSE COVID-19 
# Dataset Time series summary (csse_covid_19_time_series)
death_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
confirmed_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
recovered_df = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
country_df = pd.read_csv("https://raw.githubusercontent.com/cssegisanddata/covid-19/web-data/data/cases_country.csv")

In [ ]:
# Data preparation
death_df.head();

In [ ]:
# Data preparation
confirmed_df.head();

In [ ]:
# Data preparation
recovered_df.head();

In [ ]:
# Data preparation
country_df.head();

In [ ]:
# data cleaning - renaming
country_df.columns = map(str.lower, country_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)
death_df.columns = map(str.lower, death_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)

In [ ]:
# data cleaning - renaming
confirmed_df = confirmed_df.rename(columns = {'province/state': 'state', 'country/region': 'country'})
recovered_df = recovered_df.rename(columns = {'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns = {'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns = {'country_region': 'country'})

In [ ]:
# data cleaning - renaming
# @hidden_cell
sorted_country_df = country_df.sort_values('confirmed', ascending=False).head(5)
sorted_country_df;

In [ ]:
display(HTML('<h1>COVID-19 Confirmed/Death/Recovered cases by countries</h1>'
             '<h2>Enter number of countries you want the data for</h2>'))

In [ ]:
# Data preparation
def highlight_col(x):
    r = 'background-color: red'
    p = 'background-color: purple'
    g = 'background-color: grey'
    temp_df = pd.DataFrame('', index=x.index, columns = x.columns)
    temp_df.iloc[:,4] = p
    temp_df.iloc[:,5] = r
    temp_df.iloc[:,6] = g
    return temp_df

sorted_country_df.style.apply(highlight_col, axis=None)

In [ ]:
display(HTML('<h1>Slide to check for the worst hit countries</h1>'))

In [ ]:
# Data preparation
# x and y given as array_like objects
import plotly.express as px
fig = px.scatter(sorted_country_df.head(10), x='country', y='confirmed', size='confirmed', 
                  color='country', hover_name='country', size_max=60)
# fig.update_layout()
fig.show()

In [ ]:
display(HTML('<h1>Check the details of your country or the World</h1>'
'<h4>Enter the name of your country(in capitalized format(e.g. Italy)) and world for total cases</h4>'))

In [ ]:
import plotly.graph_objects as go

def plot_cases_for_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue', 'red']
    mode_size = [6, 8]
    line_size = [4,5]

    df_list = [confirmed_df, death_df]

    fig = go.Figure()

    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 5:].columns))
            y_data = np.sum(np.asarray(df.iloc[:, 5:]), axis=0)
        
        else:
            x_data = np.array(list(df.iloc[:, 5:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,5:]), axis=0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
                                    name=labels[i],
                                    line = dict(color=colors[i], width=line_size[i]),
                                    connectgaps=True,
                                    text = "Total " + str(labels[i]) + ": "+ str(y_data[-1])
                                ))

    fig.show()
        
# plot_cases_for_country('China')

interact(plot_cases_for_country, country='World');

In [ ]:
display(HTML('<h1>Global spread of COVID-19</h1>'))

In [ ]:
import folium

world_map = folium.Map(location=[11,0], tiles="cartodbpositron", 
                       zoom_start=2, max_zoom=6, min_zoom=2)

for i in range(len(confirmed_df)):
    folium.Circle(
    location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
              fill = True,
              radius = (int((np.log(confirmed_df.iloc[i,-1]+1.0001))) + 0.2)*50000,
              fill_color = 'blue',
              color = 'red',
              tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                            "<h4 style='text-align:center;font-weight:bold'>"+confirmed_df.iloc[i]['country'] +"</h4>"+
                            "<hr style='margin:10px;color:white;'>"+
                            "<ul style='color:white;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                            "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                            "<li>Deaths: "+str(death_df.iloc[i,-1])+"</li>"+
                            "<li>Death Rate: "+str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                        "</ul></div>",
              ).add_to(world_map)
              
world_map

In [ ]:

display(HTML('<h2>More Info on COVID-19:<a href="#More-Info-on-COVID-19:"></h2>'
'<ul>'
             '<li><a href="https://www.who.int/health-topics/coronavirus">https://www.who.int/health-topics/coronavirus</a></li>'
             '<li><a href="https://www.who.int/emergencies/diseases/novel-coronavirus-2019">https://www.who.int/emergencies/diseases/novel-coronavirus-2019</a></li>'
             '<li><a href="https://www.nature.com/articles/s41597-020-0448-0">https://www.nature.com/articles/s41597-020-0448-0</a></li>'
'</ul>'))

In [ ]:
# import nbinteract as nbi
# nbi.publish('kenaitian/voila/master', 'DataVisualization.ipynb');

In [ ]:
import matplotlib.pyplot as plt

@interact
def polynomial(split=[0,100]):
    plt.pie([split/100.0, 1.0-split/100.0])
    plt.show()

In [4]:
# First make sure that weaviate is running before you create schema
client = weaviate.Client("http://localhost:8080")
client.create_schema("https://raw.githubusercontent.com/semi-technologies/weaviate-python-client/master/documentation/getting_started/people_schema.json")

In [5]:
# Create Things or Actions
client.create_thing({"name": "John von Neumann"}, "Person", "b36268d4-a6b5-5274-985f-45f13ce0c642")
client.create_thing({"name": "Alan Turing"}, "Person", "1c9cd584-88fe-5010-83d0-017cb3fcb446")
client.create_thing({"name": "Legends"}, "Group", "2db436b5-0557-5016-9c5f-531412adf9c6")

'2db436b5-0557-5016-9c5f-531412adf9c6'

In [ ]:
# Actions can be created in a similar fashion like this:
# client.create_action({"description": "John von Neumann's book"}, "Buy")

In [6]:
# You can add cross-references through:
client.add_reference_to_thing("2db436b5-0557-5016-9c5f-531412adf9c6", "members", "b36268d4-a6b5-5274-985f-45f13ce0c642")
client.add_reference_to_thing("2db436b5-0557-5016-9c5f-531412adf9c6", "members", "1c9cd584-88fe-5010-83d0-017cb3fcb446")

In [7]:
client.create_thing({"name": "Jesse James Gardner"}, "Person", "a36268d4-a6b5-5274-985f-45f13ce0c642")

'a36268d4-a6b5-5274-985f-45f13ce0c642'